### Import libraries and datasets

In [1]:
import os
import subprocess
import glob
import shutil
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import read_gbq
from datetime import datetime
from pytz import timezone

In [2]:
folder_path = os.getcwd().replace("\\", "/")
print(f"Folder path: {folder_path}")

Folder path: c:/Users/Michael/Documents/GitHub/appStoreAnalytics/dataSources


In [3]:
# Hard-coded variables

project_id = "big-data-analytics-412816"
dataset = "practice_project"
apple_db_path = f"{project_id}.{dataset}.apple"
android_db_path = f"{project_id}.{dataset}.android"

client = bigquery.Client.from_service_account_json(f"{folder_path}/big-data-analytics-412816-1be796546c90.json")
apple_csv_path = f"{folder_path}/apple.csv"
android_csv_path = f"{folder_path}/android.csv"

In [4]:
# Apple
## Clone the repository
subprocess.run(["git", "clone", "https://github.com/gauthamp10/apple-appstore-apps.git"])
## Change directory to the dataset folder
os.chdir("apple-appstore-apps/dataset")
## Extract the tar.lzma file
subprocess.run(["tar", "-xvf", "appleAppData.json.tar.lzma"])
## Read into DataFrame
apple = pd.read_json("appleAppData.json")
print(f"Apple dataset: {apple.shape[0]} rows, {apple.shape[1]} columns")

# Android
## Clone the repository
subprocess.run(["git", "clone", "https://github.com/gauthamp10/Google-Playstore-Dataset.git"])
## Change directory to the dataset folder
os.chdir("Google-Playstore-Dataset/dataset")
## Extract all .tar.gz files
for f in os.listdir():
    if f.endswith(".tar.gz"):
        subprocess.run(["tar", "-xvf", f])
combined_csv = "Google-Playstore-Dataset.csv"
with open(combined_csv, "wb") as outfile:
    for csvfile in glob.glob("Part?.csv"):
        with open(csvfile, "rb") as infile:
            outfile.write(infile.read())
## Read into DataFrame
android = pd.read_csv("Google-Playstore-Dataset.csv")
print(f"Android dataset: {android.shape[0]} rows, {android.shape[1]} columns")

Apple dataset: 1230376 rows, 21 columns
Android dataset: 2312944 rows, 24 columns


### Push datasets into Google BigQuery

In [5]:
# Create 'apple' table in DB
job = client.query(f"DROP TABLE {apple_db_path}").result()
client.create_table(bigquery.Table(apple_db_path))

# Create 'android' table in DB
job = client.query(f"DROP TABLE {android_db_path}").result()
client.create_table(bigquery.Table(android_db_path))

Table(TableReference(DatasetReference('big-data-analytics-412816', 'practice_project'), 'android'))

In [6]:
# Save data as CSV files
apple.to_csv(apple_csv_path)
android.to_csv(android_csv_path)

In [7]:
%%time

# Push data into DB

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV,
)
apple_config = client.dataset(dataset).table('apple')
android_config = client.dataset(dataset).table('android')

with open(apple_csv_path, 'rb') as f:
    load_job = client.load_table_from_file(f, apple_config, job_config=job_config)
load_job.result()

with open(android_csv_path, 'rb') as f:
    load_job = client.load_table_from_file(f, android_config, job_config=job_config)
load_job.result() 

CPU times: total: 516 ms
Wall time: 6min 13s


LoadJob<project=big-data-analytics-412816, location=US, id=9faf5564-03de-40d0-a616-5ec8cc05ac1b>

In [8]:
# Remove CSV files and folder
try:
    os.remove(apple_csv_path)
    os.remove(android_csv_path)
    shutil.rmtree(f"{folder_path}apple-appstore-apps")
except:
    pass

In [9]:
%%time

apple_query = f"""
    SELECT *
    FROM {apple_db_path}
"""
apple_df = read_gbq(apple_query, project_id)
print(apple_df.shape)

android_query = f"""
    SELECT *
    FROM {android_db_path}
"""
android_df = read_gbq(android_query, project_id)
print(android_df.shape)

C:\Users\Michael\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


(1230376, 22)


C:\Users\Michael\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


(2312944, 25)
CPU times: total: 2min 4s
Wall time: 36min 54s


### Add dateTime table to track date and time of database update

In [62]:
dateTime_csv_path = f"{folder_path}/dateTime.csv"

# Create 'dateTime' table in DB
dateTime_db_path = f"{project_id}.{dataset}.dateTime"
client.create_table(bigquery.Table(dateTime_db_path), exists_ok = True)

Table(TableReference(DatasetReference('big-data-analytics-412816', 'practice_project'), 'dateTime'))

In [63]:
current_time = datetime.now(timezone('Asia/Shanghai'))
timestamp_string = current_time.isoformat()

In [64]:
dt = datetime.strptime(timestamp_string, '%Y-%m-%dT%H:%M:%S.%f%z')
date_time_str = dt.strftime('%d-%m-%Y %H:%M:%S')  # Date and time
time_zone = dt.strftime('%z')  # Time zone
output = f"Date and Time: {date_time_str}; Time zone: {time_zone}"

In [65]:
dateTime_df = pd.DataFrame(data = [output], columns = ['dateTime'])
dateTime_df.to_csv(f"{folder_path}/dateTime.csv", header = True)

In [67]:
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV,
)

dateTime_config = client.dataset(dataset).table('dateTime')

with open(dateTime_csv_path, 'rb') as f:
    load_job = client.load_table_from_file(f, dateTime_config, job_config=job_config)
load_job.result()

LoadJob<project=big-data-analytics-412816, location=US, id=a17247a1-44f1-4e52-a1c6-44aeb90145b7>

In [68]:
# Remove CSV file
try:
    os.remove(dateTime_csv_path)
except:
    pass